<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Jupyter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Geophysikalisches Praktikum: Modelling

## Geocampus Lankwitz
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*



In this notebook, we want to move on to make a model for the GeoCampus.

There are three tasks we need to do:

1. Get an elevation data file and possibly some information about sub-surface structure.
2. Create a forward model for gravity and magnetics.
3. Run a forward model

<img style="height: 10cm;" src="images/Campus_tunnel.jpg">

*This part needs to be filled with life, we will try it live...*


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import GP_modules
import lib_modelling
GP_modules.change_fontsize()

----
## Topography

Read elevation data from file, extract sub-region, and shift elevation to total offset with
value `ZTOP`. Load also both tunnel and rigole geometries.

In [1]:
# read elevation data
# read structure data

In [2]:
# define function to plot elevation model

----
## Gravity and magnetic anomalies

Function to sum up all elements over the elevation grid.

In [3]:
# define function for gravity and magnetics forward model

Define gravity and magnetic elements in real coordinates and calculate forward model.

In [4]:
# define test structure, calculate forward signal

In [5]:
# define function to plot forward models

----
## Model for GeoCampus Lankwitz ...

- Define start end end coordinates, `xcorner`, `ycorner`. 
- Discretise in 1m intervals, `dr=1`.
- Use Polar coordinates along linear structure:
$$
\begin{array}{rcl}
x &=& r \cos \phi \\
y &=& r \sin \phi
\end{array}
$$
and
$$
\begin{array}{rcl}
r &=& \sqrt{x^2 + y^2} \\
\tan \phi &=& \frac{y}{x}
\end{array}
$$
- With the assumption of boxes (squares) with 1m size length,
$V_b=a b c$, 
the corresponding volume of the sphere is $V_s = \frac{4}{3}\pi r^3$:
$$
r^3 = \frac{3 a b c}{4 \pi}
$$
thus $r \simeq 0.62$ m.

In [6]:
# real model, definition

In [8]:
# calculate real model

In [9]:
# plot real model

... done